[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jburgy/blog/blob/main/notebooks/Fake+Bokeh+Server+Colab.ipynb)

In [ ]:
from IPython import get_ipython
from IPython.display import publish_display_data
from bokeh.core.json_encoder import serialize_json
from bokeh.embed.elements import div_for_render_item
from bokeh.embed.util import RenderItem
from bokeh.io import curdoc
from bokeh.io.notebook import (
    EXEC_MIME_TYPE, HTML_MIME_TYPE, JS_MIME_TYPE,
    load_notebook,
)
from bokeh.protocol import Protocol
from bokeh.util.serialization import make_id
from google.colab._import_hooks._bokeh import _bokeh_loaded_in_this_cell, _post_execute
from jinja2 import ChoiceLoader, DictLoader, Environment, PackageLoader

doc_nb_js = """
{% extends "try_run.js" %}

{% block code_to_run %}
{
    const { Document } = root.Bokeh.require("document");
    const doc = Document.from_json({{ doc_json }});
    const { elementid, notebook_comms_target, root_ids } = {{ render_item }};
    const { colab: { kernel: { comms } } } = google;

    comms.open(notebook_comms_target).then(async (channel) => {
        doc.on_change((event) => {
            if (event.setter_id === notebook_comms_target) {
                return;
            }
            channel.send(doc.create_json_patch([event]));
        });

        for await (const { data } of channel.messages) {
            doc.apply_json_patch(data);
        }
    });

    const element = document.getElementById(elementid);
    const roots = root_ids.map(root_id => document.getElementById(root_id));
    root.Bokeh.embed.add_document_standalone(doc, element, roots);
}
{% endblock %}
"""

DOC_NB_JS = Environment(loader=ChoiceLoader([
    DictLoader({"doc_nb_js.js": doc_nb_js}),
    PackageLoader("bokeh.core", "_templates")
])).get_template("doc_nb_js.js")

ipython = get_ipython()


def show(obj):
    global _bokeh_loaded_in_this_cell
    if not _bokeh_loaded_in_this_cell:
        _bokeh_loaded_in_this_cell = True
        ipython.events.register('post_run_cell', _post_execute)
        load_notebook(hide_banner=True)

    doc = curdoc()
    doc.clear()
    doc.add_root(model)

    comms_target = make_id()

    render_item = RenderItem(
        docid=make_id(),
        elementid=make_id(),
        roots=[model]
    )
    div = div_for_render_item(render_item)

    render_item = render_item.to_json()
    render_item["notebook_comms_target"] = comms_target

    script = DOC_NB_JS.render(
        doc_json=serialize_json(doc.to_json()),
        render_item=serialize_json(render_item),
    )

    publish_display_data({HTML_MIME_TYPE: div})
    publish_display_data(
        {JS_MIME_TYPE: script, EXEC_MIME_TYPE: ""},
        metadata={EXEC_MIME_TYPE: {"id": obj.id}}
    )

    protocol = Protocol()

    def target_func(comm, _open_msg):
        @doc.on_change
        def dispatch(event):
            message = protocol.create('PATCH-DOC', [event])
            comm.send(message.content)

        @comm.on_msg
        def handle_msg(msg):
            doc.apply_json_patch(msg["content"]["data"])

    ipython.kernel.comm_manager.register_target(comms_target, target_func)

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure

x = [t * .005 for t in range(200)]
source = ColumnDataSource(data=dict(x=x, y=x))

def callback(attr, old, new):
    source.data = dict(x=x, y=[t**new for t in x])

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.on_change('value', callback)

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

model = column(slider, plot)
show(model)